In [9]:
from scipy.stats import pearsonr, spearmanr
import pandas as pd

In [10]:
models = ['bloom-560m', 'bloom-1b1', 'bloom-1b7']
layers = ['inter-layer-13', 'inter-layer-17']

In [11]:
avg_similarities = []
avg_overlap_rates = []
prws_similarities = []
prws_overlap_rates = []

for model in models:
    for layer in layers:
        df_similarity = pd.read_csv(f'csv_files/{model}_{layer}_cos-similarity.csv', index_col=0)
        df_ovlp = pd.read_csv(f'csv_files/{model}_{layer}_f1-score.csv', index_col=0)
        
        ckpts = df_similarity.columns
        langs = df_similarity.index
        for ckpt in ckpts:
            similarity_list = []
            ovlp_list = []
            for lang in langs:
                if lang == 'code':
                    continue
                similarity_list.append(float(df_similarity[ckpt][lang]))
                ovlp_list.append(float(df_ovlp[ckpt][lang]))
            
            prws_similarities += similarity_list
            prws_overlap_rates += ovlp_list
            avg_similarities.append(sum(similarity_list)/len(similarity_list))
            avg_overlap_rates.append(sum(ovlp_list)/len(ovlp_list))


In [12]:
print("avg similarities: ", avg_similarities)
print("avg overlap rates: ", avg_overlap_rates)

pearson_corr, pearson_p_value = pearsonr(avg_similarities, avg_overlap_rates)
print('Pearsons correlation: %.3f' % pearson_corr)
print(f'Pearsons P value: {pearson_p_value}')


spearman_corr, spearman_p_value = spearmanr(avg_similarities, avg_overlap_rates)
print('Spearmans correlation: %.3f' % spearman_corr)
print(f'Spearmans P value: {spearman_p_value}')

print("pairwise similarities: ", prws_similarities)
print("pairwise overlap rates: ", prws_overlap_rates)

pearson_corr, pearson_p_value = pearsonr(prws_similarities, prws_overlap_rates)
print('Pearsons correlation: %.3f' % pearson_corr)
print(f'Pearsons P value: {pearson_p_value}')


spearman_corr, spearman_p_value = spearmanr(prws_similarities, prws_overlap_rates)
print('Spearmans correlation: %.3f' % spearman_corr)
print(f'Spearmans P value: {spearman_p_value}')



avg similarities:  [0.835360499013147, 0.7551146915124463, 0.853988929643149, 0.8639143534799827, 0.8583227795231596, 0.901456389390068, 0.8776431258467953, 0.853988929643149, 0.7128053139735265, 0.8479067387138977, 0.7414326052334771, 0.8821279270624148, 0.8730091208732114, 0.8579685349811766, 0.9054689644510895, 0.8918115336788358, 0.8821279270624148, 0.7007252566800238, 0.8225500563786418, 0.8412136286360622, 0.8788327644006737, 0.8987883598303186, 0.916353109340506, 0.8946056937009269, 0.6894626611236114, 0.8495544485354412, 0.8206446872770815, 0.841856498121969, 0.7827848640962434, 0.877735279322636, 0.893086637019556, 0.9156428973592106, 0.8970089978202161, 0.6766680263245028, 0.865887753374959, 0.8418070581777939, 0.8455565328940586, 0.8700709760868711, 0.9364126493454443, 0.9310081605825857, 0.9217433570182205, 0.8943597526015287, 0.8820090830035532, 0.8562637143378, 0.8162448319562645, 0.923076396539855, 0.9248273495091544, 0.9185132489371566, 0.8960991286052367, 0.88733625009